In [5]:
import pandas as pd
import numpy as np
import re

In [6]:
df = pd.read_csv("../Data/Raw/GoodReads_500.csv")
df_api = pd.read_csv("../Data/Raw/Googlebooks.csv")

# 1. Cleaning GoodReads Scraped Data 

In [8]:
df.head(3)

,Unnamed: 0,Rank,Title,Author,Avg_Rating,Num_Ratings,ISBN,ASIN,Format_and_Page,Genres
0,0,1,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,4.34,9318770,9780439023481,0439023483,"374 pages, Hardcover",Genres\nYoung Adult\nFiction\nFantasy\nScience...
1,1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,4.50,3616863,Not found,0439686520,"912 pages, Paperback",Genres\nFantasy\nYoung Adult\nFiction\nMagic\n...
2,2,3,Pride and Prejudice,Jane Austen,4.29,4517843,9781441341709,1441341706,"279 pages, Paperback",Genres\nFiction\nHistorical Fiction\nHistorica...


In [9]:
# 1. Check for missing/null values in key columns and flag these rows for manual review.
#    Create a new column 'Review_Flag' that indicates if any key field is missing.
key_columns = ['Title', 'Author', 'Avg_Rating', 'Genres']
df['Review_Flag'] = df[key_columns].isnull().any(axis=1)

missing_key_col = df[df['Review_Flag']]

In [10]:
# 2. Remove duplicate rows. (Assuming duplicate rows have the same values in all columns.)
df.drop_duplicates(inplace=True)

In [11]:
# Function to extract text within parentheses and store it in the Series column.
def extract_series(title):
    # This regex captures text between '(' and ')'
    match = re.search(r'\((.*?)\)', title)
    if match:
        return match.group(1).strip()
    return ''

# Function to remove the parentheses and their contents from the title.
def clean_title(title):
    return re.sub(r'\s*\(.*?\)', '', title).strip()

# Create the Series column from the Title column.
df['Series'] = df['Title'].apply(extract_series)
# Clean the Title column by removing the extracted text.
df['Title'] = df['Title'].apply(clean_title)

In [12]:
# 4. Split the 'Format_And_Page' column into 'Page_Number' and 'Format'
#    Example formats: "232 pages, paperback" or "232, paperback". We'll extract digits as page number and the rest as format.
def split_format_and_page(text):
    # Ensure the text is a string (handle missing or non-string data)
    if pd.isnull(text):
        return pd.NA, pd.NA
    # Extract page number (digits)
    page_match = re.search(r'(\d+)', text)
    page_number = page_match.group(1) if page_match else pd.NA
    # Remove the page number and non-alpha characters at the beginning to extract the format.
    # This approach assumes that after the number there is some text indicating format.
    format_text = re.sub(r'\d+', '', text)
    # Remove common words like "pages" and punctuation
    format_text = re.sub(r'pages?', '', format_text, flags=re.IGNORECASE)
    format_text = format_text.replace(',', '').strip()
    return page_number, format_text

df[['Page_Number', 'Format']] = df['Format_and_Page'].apply(lambda x: pd.Series(split_format_and_page(x)))


In [13]:
df.head()

,Unnamed: 0,Rank,Title,Author,Avg_Rating,Num_Ratings,ISBN,ASIN,Format_and_Page,Genres,Review_Flag,Series,Page_Number,Format
0,0,1,The Hunger Games,Suzanne Collins,4.34,9318770,9780439023481,0439023483,"374 pages, Hardcover",Genres\nYoung Adult\nFiction\nFantasy\nScience...,False,"The Hunger Games, #1",374,Hardcover
1,1,2,Harry Potter and the Order of the Phoenix,J.K. Rowling,4.50,3616863,Not found,0439686520,"912 pages, Paperback",Genres\nFantasy\nYoung Adult\nFiction\nMagic\n...,False,"Harry Potter, #5",912,Paperback
2,2,3,Pride and Prejudice,Jane Austen,4.29,4517843,9781441341709,1441341706,"279 pages, Paperback",Genres\nFiction\nHistorical Fiction\nHistorica...,False,,279,Paperback
3,3,4,To Kill a Mockingbird,Harper Lee,4.26,6564282,9780060935467,0060935464,"323 pages, Paperback",Genres\nFiction\nHistorical Fiction\nSchool\nL...,False,,323,Paperback
4,4,5,The Book Thief,Markus Zusak,4.39,2746358,Not found,0375831002,"592 pages, Kindle Edition",Genres\nHistorical Fiction\nFiction\nYoung Adu...,False,,592,Kindle Edition


In [14]:
# 5. Clean the 'Genres' column: Remove newline characters and ensure they are comma-separated.
df['Genres'] = df['Genres'].astype(str).replace(r'\n', ', ', regex=True).str.strip()

In [15]:
df = df.drop(['Unnamed: 0', 'Format_and_Page', 'Review_Flag'], axis=1)

In [16]:
#Casting avg rating to float and page number to integer
df['Avg_Rating'] = pd.to_numeric(df['Avg_Rating'], errors='coerce')
df['Page_Number'] = pd.to_numeric(df['Page_Number'], errors='coerce').astype('Int64')

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512 entries, 0 to 511
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         512 non-null    int64  
 1   Title        512 non-null    object 
 2   Author       512 non-null    object 
 3   Avg_Rating   502 non-null    float64
 4   Num_Ratings  512 non-null    int64  
 5   ISBN         512 non-null    object 
 6   ASIN         512 non-null    object 
 7   Genres       512 non-null    object 
 8   Series       512 non-null    object 
 9   Page_Number  512 non-null    Int64  
 10  Format       512 non-null    object 
dtypes: Int64(1), float64(1), int64(2), object(7)
memory usage: 44.6+ KB


# 2. Cleaning Google API Data

In [19]:
df_api.head()

,Book Title,Authors,Rating,ISBN,ASIN,Number of Pages,Format,Genres
0,A Game of Thrones,George R. R. Martin,4.5,NaN,NaN,876.0,paperback,Fiction
1,The Invisible Man,"Len Jenkin, Herbert George Wells",NaN,9.780871e+12,NaN,78.0,paperback,Drama
2,Crime and Punishment,Fyodor Dostoyevsky,NaN,NaN,NaN,520.0,paperback,Fiction
3,Cards on the Table,Agatha Christie,NaN,9.780006e+12,NaN,228.0,paperback,Fiction
4,4.50 from Paddington,Agatha Christie,NaN,9.780006e+12,NaN,228.0,paperback,Fiction


In [20]:
# 1. Check for missing/null values in key columns and flag these rows for manual review.
#    Create a new column 'Review_Flag' that indicates if any key field is missing.
key_columns = ['Book Title', 'Authors', 'Genres']
df_api['Review_Flag'] = df_api[key_columns].isnull().any(axis=1)

missing_key_col = df_api[df_api['Review_Flag']]

df_api = df_api[df_api['Review_Flag'] == False]


In [21]:
# 2. Remove books with less than 20 pages

df_api = df_api[df_api['Number of Pages'] >= 20]


In [22]:
df_api.info()

<class 'pandas.core.frame.DataFrame'>
Index: 797 entries, 0 to 1000
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Book Title       797 non-null    object 
 1   Authors          797 non-null    object 
 2   Rating           118 non-null    float64
 3   ISBN             604 non-null    float64
 4   ASIN             0 non-null      float64
 5   Number of Pages  797 non-null    float64
 6   Format           797 non-null    object 
 7   Genres           797 non-null    object 
 8   Review_Flag      797 non-null    bool   
dtypes: bool(1), float64(4), object(4)
memory usage: 56.8+ KB


In [23]:
df_api = df_api.drop(['ASIN','Review_Flag'], axis=1)

In [24]:
df_api.info()

<class 'pandas.core.frame.DataFrame'>
Index: 797 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Book Title       797 non-null    object 
 1   Authors          797 non-null    object 
 2   Rating           118 non-null    float64
 3   ISBN             604 non-null    float64
 4   Number of Pages  797 non-null    float64
 5   Format           797 non-null    object 
 6   Genres           797 non-null    object 
dtypes: float64(3), object(4)
memory usage: 49.8+ KB


In [25]:
# 2. Remove duplicate rows. (Assuming duplicate rows have the same values in all columns.)
df_api.drop_duplicates(inplace=True)

In [26]:
df_api = df_api.drop_duplicates(subset=['Book Title'], keep='first')

In [27]:
df_api

,Book Title,Authors,Rating,ISBN,Number of Pages,Format,Genres
0,A Game of Thrones,George R. R. Martin,4.5,NaN,876.0,paperback,Fiction
1,The Invisible Man,"Len Jenkin, Herbert George Wells",NaN,9.780871e+12,78.0,paperback,Drama
2,Crime and Punishment,Fyodor Dostoyevsky,NaN,NaN,520.0,paperback,Fiction
3,Cards on the Table,Agatha Christie,NaN,9.780006e+12,228.0,paperback,Fiction
4,4.50 from Paddington,Agatha Christie,NaN,9.780006e+12,228.0,paperback,Fiction
...,...,...,...,...,...,...,...
995,The Children's Crusade,Henry Treece,NaN,NaN,193.0,paperback,Crusades
996,The Pleasures of Love,Jean Plaidy,NaN,9.780745e+12,437.0,paperback,Great Britain
997,Escape to Eaglehawk,Vashti Farrer,NaN,9.781864e+12,125.0,paperback,historical fiction
999,When a Girl is Born,Pamela Grant,NaN,9.780193e+12,136.0,paperback,China


In [28]:
# Rename columns in the Google file to match the GoodReads structure.
# Adjust the mappings based on your actual column names.
df_api.rename(columns={
    'Book Title':'Title',
    'Authors':'Author',
    'Rating':'Avg_Rating',
    'Number of Pages':'Page_Number' 
}, inplace=True)

df_api

,Title,Author,Avg_Rating,ISBN,Page_Number,Format,Genres
0,A Game of Thrones,George R. R. Martin,4.5,NaN,876.0,paperback,Fiction
1,The Invisible Man,"Len Jenkin, Herbert George Wells",NaN,9.780871e+12,78.0,paperback,Drama
2,Crime and Punishment,Fyodor Dostoyevsky,NaN,NaN,520.0,paperback,Fiction
3,Cards on the Table,Agatha Christie,NaN,9.780006e+12,228.0,paperback,Fiction
4,4.50 from Paddington,Agatha Christie,NaN,9.780006e+12,228.0,paperback,Fiction
...,...,...,...,...,...,...,...
995,The Children's Crusade,Henry Treece,NaN,NaN,193.0,paperback,Crusades
996,The Pleasures of Love,Jean Plaidy,NaN,9.780745e+12,437.0,paperback,Great Britain
997,Escape to Eaglehawk,Vashti Farrer,NaN,9.781864e+12,125.0,paperback,historical fiction
999,When a Girl is Born,Pamela Grant,NaN,9.780193e+12,136.0,paperback,China


In [29]:
# Add any missing columns to google_df with empty values so both DataFrames have the same columns.
for col in df.columns:
    if col not in df_api.columns:
        df_api[col] = ''

# Ensure both DataFrames have the same column order
df_api = df_api[df.columns]

In [30]:
df_api.info()

<class 'pandas.core.frame.DataFrame'>
Index: 708 entries, 0 to 1000
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         708 non-null    object 
 1   Title        708 non-null    object 
 2   Author       708 non-null    object 
 3   Avg_Rating   100 non-null    float64
 4   Num_Ratings  708 non-null    object 
 5   ISBN         537 non-null    float64
 6   ASIN         708 non-null    object 
 7   Genres       708 non-null    object 
 8   Series       708 non-null    object 
 9   Page_Number  708 non-null    float64
 10  Format       708 non-null    object 
dtypes: float64(3), object(8)
memory usage: 66.4+ KB


In [31]:
df_api['ISBN'] = pd.to_numeric(df_api['ISBN'], errors='coerce').astype('Int64')

In [32]:
#Casting avg rating to float and page number to integer
df_api['Rank'] = pd.to_numeric(df_api['Rank'], errors='coerce').astype('Int64')
df_api['Num_Ratings'] = pd.to_numeric(df_api['Num_Ratings'], errors='coerce').astype('Int64')
df_api['ISBN'] = df_api['ISBN'].astype('object')
df_api['Page_Number'] = pd.to_numeric(df_api['Page_Number'], errors='coerce').astype('Int64')

In [33]:
#Saving the Cleaned Separate Datasets Before Merge
df_api.to_csv("../Data/Clean/Google_API_Clean.csv")
df.to_csv("../Data/Clean/GoodReads_Clean.csv")

In [34]:
# Concatenate the two DataFrames
combined_df = pd.concat([df, df_api], ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('combined_books.csv', index=False)

In [35]:
combined_df

,Rank,Title,Author,Avg_Rating,Num_Ratings,ISBN,ASIN,Genres,Series,Page_Number,Format
0,1,The Hunger Games,Suzanne Collins,4.34,9318770,9780439023481,0439023483,"Genres, Young Adult, Fiction, Fantasy, Science...","The Hunger Games, #1",374,Hardcover
1,2,Harry Potter and the Order of the Phoenix,J.K. Rowling,4.50,3616863,Not found,0439686520,"Genres, Fantasy, Young Adult, Fiction, Magic, ...","Harry Potter, #5",912,Paperback
2,3,Pride and Prejudice,Jane Austen,4.29,4517843,9781441341709,1441341706,"Genres, Fiction, Historical Fiction, Historica...",,279,Paperback
3,4,To Kill a Mockingbird,Harper Lee,4.26,6564282,9780060935467,0060935464,"Genres, Fiction, Historical Fiction, School, L...",,323,Paperback
4,5,The Book Thief,Markus Zusak,4.39,2746358,Not found,0375831002,"Genres, Historical Fiction, Fiction, Young Adu...",,592,Kindle Edition
...,...,...,...,...,...,...,...,...,...,...,...
1215,<NA>,The Children's Crusade,Henry Treece,NaN,<NA>,<NA>,,Crusades,,193,paperback
1216,<NA>,The Pleasures of Love,Jean Plaidy,NaN,<NA>,9780745175287,,Great Britain,,437,paperback
1217,<NA>,Escape to Eaglehawk,Vashti Farrer,NaN,<NA>,9781864290295,,historical fiction,,125,paperback
1218,<NA>,When a Girl is Born,Pamela Grant,NaN,<NA>,9780192716996,,China,,136,paperback


In [36]:
combined_df = combined_df.drop_duplicates(subset=['ISBN'], keep='first')

In [37]:
combined_df.rename(columns={
    'Rank':'Good_Reads_Popularity_Rank'
}, inplace=True)

/var/folders/s4/7wsslyw16w9bqtllmwg3nfbm0000gp/T/ipykernel_59448/2195735635.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df.rename(columns={


In [38]:
combined_df.reset_index(drop=True,inplace=True)

In [40]:
combined_df['Genres'].unique

<bound method Series.unique of 0      Genres, Young Adult, Fiction, Fantasy, Science...
1      Genres, Fantasy, Young Adult, Fiction, Magic, ...
2      Genres, Fiction, Historical Fiction, Historica...
3      Genres, Fiction, Historical Fiction, School, L...
4      Genres, Fantasy, Young Adult, Romance, Fiction...
                             ...                        
932                                              Fiction
933                                              Fiction
934                                        Great Britain
935                                   historical fiction
936                                                China
Name: Genres, Length: 937, dtype: object>

In [86]:
combined_df['Genres'] = combined_df['Genres'].str.replace("Genres,", "", regex=False).str.strip()

/var/folders/s4/7wsslyw16w9bqtllmwg3nfbm0000gp/T/ipykernel_59448/1660065618.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['Genres'] = combined_df['Genres'].str.replace("Genres,", "", regex=False).str.strip()


In [88]:
combined_df['Genres'].unique

<bound method Series.unique of 0      Young Adult, Fiction, Fantasy, Science Fiction...
1      Fantasy, Young Adult, Fiction, Magic, Audioboo...
2      Fiction, Historical Fiction, Historical, Liter...
3      Fiction, Historical Fiction, School, Literatur...
4      Fantasy, Young Adult, Romance, Fiction, Vampir...
                             ...                        
932                                              Fiction
933                                              Fiction
934                                        Great Britain
935                                   historical fiction
936                                                China
Name: Genres, Length: 937, dtype: object>

In [90]:
#Saving the final dataset
combined_df.to_csv("../Data/Clean/Combined_Book_Data.csv")